<a href="https://colab.research.google.com/github/sofia4009/Oral-disease/blob/main/Train_KaggleDS_GoogleDS_Test_GoogleDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Sofia
Date: March 26
Subject: Training and getting experience with deep learning
Classification of dataset from kaggle:  https://www.kaggle.com/datasets/salmansajid05/oral-diseases
  - use stratified k-fold cross validation (with k = 5) to split dataset A
  - use the following deep neural networks for training, testing and comparing their performance
  - (do not use any data augmentation for now; just normalise all image pixel values to the range [0, 1];
  - resize all images to dimensions 112x112x3 for speeding up the training;
  - use categorical cross entropy as the loss function and f1 score as evaluation metric;
  - use the pretrained models on ImageNet):
    - ResNet18, ResNet50, ConvNeXt, EfficientNetB0, Transformers (i.e., ViT)

#Data Preparation

In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import cv2

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as xlImage
from openpyxl import Workbook
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import h5py
import time


#Dataset Kaggle

In [ ]:
# Initializations
image_size = (112,112)

# Specify the root directory where your images are located
main_DSA_directory = '/content/oral-diseases'

# Specify the output directory to save the processed images
Preprocessed_DSA_directory = '/content/decreased_oral_diseases'

# Specify the output directory to save the excel file
results_directory = '/content/drive/My Drive/QM/results.xlsx'
Plot_directory = '/content/drive/My Drive/QM/plots.xlsx'

In [ ]:
# Install kaggle API client
!pip install -q kaggle

from google.colab import drive
drive.mount('/content/drive')

from google.colab import files

files.upload() #To prompt to upload the kaggle.json

# kaggle API client expects the file to be in ~/.kaggle
# so move it there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# we need to set permissions
!chmod 600 /root/.kaggle/kaggle.json

# check your directory before downloading the datasets
!pwd

# download the required dataset from kaggle
!kaggle datasets download -d salmansajid05/oral-diseases

#If your file is a zip file you can unzip with the following code
import zipfile

with zipfile.ZipFile('oral-diseases.zip', "r") as z:
    z.extractall(main_DSA_directory)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving kaggle.json to kaggle (1).json
/content
oral-diseases.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Normalizing them to [0,1], Resizing to 112,112, 3
def normalize_and_resize_image(image, target_size):

    # Resize image into 112*112*3
    image = image.resize(target_size)

    # Normalize pixel values to the range [0, 1]
    image = np.array(image)
    image = image / 255.0

    # Convert the normalized numpy array back to PIL image
    image = Image.fromarray((image * 255).astype(np.uint8))

    return image

# Saving the normalized images into a new directory in Google colab with the same subdirectories and structure
def process_images_in_directory(directory, Preprocessed_DS_directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file has an image extension
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                # Construct the full path to the input image file
                image_path = os.path.join(root, file)
                if 'augmented' not in image_path and 'Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset' not in image_path:
                  # Open the image using PIL
                  image = Image.open(image_path)

                  # Ensure image is in RGB mode
                  image = image.convert("RGB")

                  # Normalize and Resize the image
                  #print(image_path)
                  processed_image = normalize_and_resize_image(image, image_size)

                  # Construct the full path to the output directory
                  output_subdirectory = os.path.relpath(root, directory)
                  output_path = os.path.join(Preprocessed_DS_directory, output_subdirectory)
                  os.makedirs(output_path, exist_ok=True)

                  # Save the processed image
                  filename = os.path.join(output_path, file)
                  processed_image.save(filename, format='JPEG')  # Change 'JPEG' to the desired format
                  #print(filename)
                  #if filename.find('augmented') == -1 or filename.find('Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset') == -1:
                  #np.save(filename, image)

In [ ]:
# Call the function to process images in the directory
process_images_in_directory(main_DSA_directory, Preprocessed_DSA_directory)

In [ ]:
file_count = sum(len(files) for _, _, files in os.walk(main_DSA_directory))
print(f"Number of Original Images: {file_count}")

file_count = sum(len(files) for _, _, files in os.walk(Preprocessed_DSA_directory))
print(f"Number of Preprocessed Images: {file_count}")

Number of Original Images: 15373
Number of Preprocessed Images: 5563


In [ ]:
# Define transformations by Composing several transforms together
transform = transforms.Compose([
    transforms.Resize(image_size),
    #Convert a PIL Image or ndarray to tensor
    transforms.ToTensor()
])

# Use ImageFolder to load your dataset
dataset1 = torchvision.datasets.ImageFolder(root = Preprocessed_DSA_directory, transform=transform)

# Define number of classes and list of labels in the dataset
num_classes1 = len(dataset1.classes)
classes1 = dataset1.classes

# Define k-fold for cross-validation
k_folds = 5

# Seed (random_state) is set to initialize the random number generator while splitting the DataSet into k folds
skf_ds1 = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Define hyperparameters to search over
learning_rates = [0.0001] #
batch_sizes = [32]#128,
optimizers = [ 'Adam'] #'SGD', 'RMSprop'

epochs = 25

best_model = None
best_f1 = 0.0
best_hyperparameters = None

# Define the path where you want to save the model weights in Google Drive
checkpoint_path = '/content/drive/My Drive/QM/model.h5'


#Dataset Attached

In [ ]:
# Initializations
image_size = (112,112)

# Specify the root directory where your images are located
main_DSB_directory = '/content/New_Dataset' #'/content/oral-diseases'

# Specify the output directory to save the processed images
Preprocessed_DSB_directory = '/content' #'/content/decreased_oral_diseases'

# Specify the output directory to save the excel file
results_directory = '/content/drive/My Drive/QM/results.xlsx'
Plot_directory = '/content/drive/My Drive/QM/plots.xlsx'

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

with zipfile.ZipFile('/content/drive/MyDrive/QM/New_Dataset.zip') as zipObj:
  members = [file for file in zipObj.namelist() if "__MACOSX" not in file]
  zipObj.extractall('/content/New_Dataset', members=members)

# Optionally, remove the __MACOSX directory if it was extracted
macosx_folder = os.path.join('/content/New_Dataset', "__MACOSX")
if os.path.exists(macosx_folder):
    os.rmdir(macosx_folder)


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Normalizing them to [0,1], Resizing to 112,112, 3
def normalize_and_resize_image(image, target_size):

    # Resize image into 112*112*3
    image = image.resize(target_size)

    # Normalize pixel values to the range [0, 1]
    image = np.array(image)
    image = image / 255.0

    # Convert the normalized numpy array back to PIL image
    image = Image.fromarray((image * 255).astype(np.uint8))

    return image

# Saving the normalized images into a new directory in Google colab with the same subdirectories and structure
def process_images_in_directory(directory, Preprocessed_DS_directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file has an image extension
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                # Construct the full path to the input image file
                image_path = os.path.join(root, file)
                if 'Decay' not in image_path and 'Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset' not in image_path:
                  # Open the image using PIL
                  image = Image.open(image_path)

                  # Ensure image is in RGB mode
                  image = image.convert("RGB")

                  # Normalize and Resize the image
                  #print(image_path)
                  processed_image = normalize_and_resize_image(image, image_size)

                  # Construct the full path to the output directory
                  output_subdirectory = os.path.relpath(root, directory)
                  output_path = os.path.join(Preprocessed_DS_directory, output_subdirectory)
                  os.makedirs(output_path, exist_ok=True)

                  # Save the processed image
                  filename = os.path.join(output_path, file)
                  processed_image.save(filename, format='JPEG')  # Change 'JPEG' to the desired format
                  #print(filename)
                  #if filename.find('augmented') == -1 or filename.find('Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset') == -1:
                  #np.save(filename, image)

In [ ]:
# Call the function to process images in the directory
process_images_in_directory(main_DSB_directory, Preprocessed_DSB_directory)

In [ ]:
Preprocessed_DSB_directory = '/content/DataSet B'

file_count = sum(len(files) for _, _, files in os.walk(main_DSB_directory))
print(f"Number of Original Images: {file_count}")

file_count = sum(len(files) for _, _, files in os.walk(Preprocessed_DSB_directory))
print(f"Number of Preprocessed Images: {file_count}")

Number of Original Images: 539
Number of Preprocessed Images: 532


In [ ]:
# Define transformations by Composing several transforms together
transform = transforms.Compose([
    transforms.Resize(image_size),
    #Convert a PIL Image or ndarray to tensor
    transforms.ToTensor()
])

Preprocessed_DSB_directory = '/content/DataSet B'

# Use ImageFolder to load your dataset
dataset2 = torchvision.datasets.ImageFolder(root = Preprocessed_DSB_directory, transform=transform)

# Define number of classes and list of labels in the dataset
num_classes2 = len(dataset2.classes)
classes2 = dataset2.classes

# Define k-fold for cross-validation
k_folds = 5

# Seed (random_state) is set to initialize the random number generator while splitting the DataSet into k folds
skf_ds2 = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Define hyperparameters to search over
learning_rates = [0.0001] #
batch_sizes = [32]#128,
optimizers = [ 'Adam'] #'SGD', 'RMSprop'

epochs = 20

best_model = None
best_f1 = 0.0
best_hyperparameters = None

# Define the path where you want to save the model weights in Google Drive
checkpoint_path = '/content/drive/My Drive/QM/combined_model_fold1.h5'


# Training the dataset Using Pre-trained models on ImageNet

#ResNet50

In [ ]:
from torchvision.models import resnet50

In [ ]:
# Save the results as a DataFrame to be saved in an Excel file
def Save_Results_to_Drive(results, results_directory):
    df = pd.DataFrame(results)

    # Check if the file exists
    if os.path.isfile(results_directory):
        # Load an existing Excel file
        existing_file = pd.read_excel(results_directory)
        # Append the new DataFrame to the existing file
        df = pd.concat([existing_file, df])

        # Remove the existing file
        os.remove(results_directory)

    # Write the DataFrame to the Excel file in the directory
    df.to_excel(results_directory, index=False)

    return df

In [ ]:
# An empty list for storing the results
checkpoint_path = '/content/drive/My Drive/QM/combined_model_fold2.h5'

results = []
f1_vals = [0] * k_folds
f1_vals1 = [0] * k_folds
f1_vals2 = [0] * k_folds
results.append({'Model': 'ResNet50'})

# Initialize lists to store training and validation losses and accuracies
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

val_losses1 = []
val_accuracies1 = []

val_losses2 = []
val_accuracies2 = []

num_epochs = 20
counter = 0

for lr in learning_rates:
    for batch_size in batch_sizes:
      for optimizer_name in optimizers:
        print(f"Training with optimizer: {optimizer_name}, learning rate: {lr}, batch size: {batch_size}")
        for fold, ((train1_idx, val1_idx), (train2_idx, val2_idx)) in enumerate(zip(skf_ds1.split(dataset1.imgs, dataset1.targets), skf_ds2.split(dataset2.imgs, dataset2.targets))):
            # Combine indices for training and validation
            combined_train_idx = np.concatenate((train1_idx, train2_idx))
            combined_val_idx = np.concatenate((val1_idx, val2_idx))

            # Create SubsetRandomSampler for combined indices
            train_sampler = torch.utils.data.SubsetRandomSampler(combined_train_idx)
            val_sampler = torch.utils.data.SubsetRandomSampler(combined_val_idx)
            val_sampler1 = torch.utils.data.SubsetRandomSampler(val1_idx)
            val_sampler2 = torch.utils.data.SubsetRandomSampler(val2_idx)

            # Create DataLoader for training and validation
            combined_train_dataset = torch.utils.data.ConcatDataset([dataset1, dataset2])
            combined_val_dataset = torch.utils.data.ConcatDataset([dataset1, dataset2])

            # Create DataLoader for training and validation
            train_loader = torch.utils.data.DataLoader(combined_train_dataset, batch_size=batch_size, sampler=train_sampler)
            val_loader = torch.utils.data.DataLoader(combined_val_dataset, batch_size=batch_size, sampler=val_sampler)
            val_loader1 = torch.utils.data.DataLoader(dataset1, batch_size=batch_size, sampler=val_sampler1)
            val_loader2 = torch.utils.data.DataLoader(dataset2, batch_size=batch_size, sampler=val_sampler2)

            best_f1 = 0
            best1_f1 = 0
            best2_f1 = 0

            # Define the device (GPU if available, otherwise CPU)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            # Define the neural network
            model = resnet50(weights='ResNet50_Weights.DEFAULT')
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, num_classes1)
            model = model.to(device)

            # Load model
            #model.load_state_dict(torch.load(checkpoint_path))

            # Define loss function
            criterion = nn.CrossEntropyLoss()

            # Define optimizer
            if optimizer_name == 'Adam':
                optimizer = optim.Adam(model.parameters(), lr=lr)
            elif optimizer_name == 'SGD':
                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
            elif optimizer_name == 'RMSprop':
                optimizer = optim.RMSprop(model.parameters(), lr=lr)

            # Train and evaluate
            for epoch in range(num_epochs):
                print('start of epoch', str(epoch + 1))
                # Training loop
                start_time = time.time()
                model.train()
                running_loss = 0.0
                correct = 0
                total = 0
                i = 0

                for inputs, labels in train_loader:
                    inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()
                    _, preds = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (preds == labels).sum().item()
                epoch_train_loss = running_loss / len(train_loader)
                epoch_train_accuracy = correct / total
                train_losses.append(epoch_train_loss)
                train_accuracies.append(epoch_train_accuracy)

                # Validation loop
                model.eval()
                all_preds = []
                all_labels = []
                running_loss = 0.0
                correct = 0
                total = 0

                all_preds1 = []
                all_labels1 = []
                running_loss1 = 0.0
                correct1 = 0
                total1 = 0

                all_preds2 = []
                all_labels2 = []
                running_loss2 = 0.0
                correct2 = 0
                total2 = 0

                with torch.no_grad():
                  for inputs, labels in val_loader:
                      inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
                      outputs = model(inputs)
                      loss = criterion(outputs, labels)
                      running_loss += loss.item()
                      _, preds = torch.max(outputs, 1)
                      total += labels.size(0)
                      correct += (preds == labels).sum().item()
                      all_preds.extend(preds.cpu().numpy())
                      all_labels.extend(labels.cpu().numpy())
                  # The duration of time needed for one fold
                  end_time = time.time()
                  duration = end_time - start_time


                  for inputs, labels in val_loader1:
                      inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
                      outputs = model(inputs)
                      loss = criterion(outputs, labels)
                      running_loss1 += loss.item()
                      _, preds = torch.max(outputs, 1)
                      total1 += labels.size(0)
                      correct1 += (preds == labels).sum().item()
                      all_preds1.extend(preds.cpu().numpy())
                      all_labels1.extend(labels.cpu().numpy())
                  # The duration of time needed for one fold
                  end_time1 = time.time()
                  duration1 = end_time1 - start_time

                  for inputs, labels in val_loader2:
                      inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
                      outputs = model(inputs)
                      loss = criterion(outputs, labels)
                      running_loss2 += loss.item()
                      _, preds = torch.max(outputs, 1)
                      total2 += labels.size(0)
                      correct2 += (preds == labels).sum().item()
                      all_preds2.extend(preds.cpu().numpy())
                      all_labels2.extend(labels.cpu().numpy())
                  # The duration of time needed for one fold
                  end_time2 = time.time()
                  duration2 = end_time2 - start_time

                  epoch_val_loss = running_loss / len(val_loader)
                  epoch_val_accuracy = correct / total
                  val_losses.append(epoch_val_loss)
                  val_accuracies.append(epoch_val_accuracy)

                  epoch_val_loss1 = running_loss1 / len(val_loader1)
                  epoch_val_accuracy1 = correct1 / total1
                  val_losses1.append(epoch_val_loss1)
                  val_accuracies1.append(epoch_val_accuracy1)

                  epoch_val_loss2 = running_loss2 / len(val_loader2)
                  epoch_val_accuracy2 = correct2 / total2
                  val_losses2.append(epoch_val_loss2)
                  val_accuracies2.append(epoch_val_accuracy2)

                  # Calculate F1 score
                  f1 = f1_score(all_labels, all_preds, average='macro')
                  print(f"Both DS: Fold {fold+1}, Epoch {epoch+1}, F1 Score: {f1}, Duration: {duration} s")

                  # Calculate F1 score dataset1
                  f1_1 = f1_score(all_labels1, all_preds1, average='macro')
                  print(f"Kaggle: Fold {fold+1}, Epoch {epoch+1}, F1 Score: {f1_1}, Duration: {duration1} s")

                  # Calculate F1 score dataset2
                  f1_2 = f1_score(all_labels2, all_preds2, average='macro')
                  print(f"Google: Fold {fold+1}, Epoch {epoch+1}, F1 Score: {f1_2}, Duration: {duration2} s")

                  # Update best F1 score and model
                  if f1 > best_f1:
                    best_f1 = f1

                  # Update best F1 score and model dataset1
                  if f1_1 > best1_f1:
                    best1_f1 = f1_1

                  # Update best F1 score and model dataset2
                  if f1_2 > best2_f1:
                    best2_f1 = f1_2

            print('end of epoch: ', str(epoch+1))
            f1_vals[fold] = best_f1
            f1_vals1[fold] = best1_f1
            f1_vals2[fold] = best2_f1

            results.append({
                    'Optimizer': optimizer_name,
                    'batch_size': batch_size,
                    'learning_rate': lr,
                    'Fold': fold+1,
                    'F1 Score': best_f1,
                    'Duration': duration,
                    'F1 Score Kaggle': best1_f1,
                    'Duration': duration1,
                    'F1 Score Google': best2_f1,
                    'Duration': duration2
              })

            df = Save_Results_to_Drive(results , results_directory)

            # Define a ModelCheckpoint callback to save the weights
            #path = checkpoint_path + 'model' + int(fold+1) + '.h5'
            checkpoint_callback = ModelCheckpoint(filepath= checkpoint_path,
                                      save_weights_only=True,
                                      monitor='val_loss',
                                      mode='min',
                                      save_best_only=True)

            torch.save(model.state_dict(), checkpoint_path)
            #model.save(checkpoint_path, save_format="h5")

        results.append({
                  'min f1': min(f1_vals),
                  'max f1': max(f1_vals),
                  'average f1': sum(f1_vals)/len(f1_vals)
                  })
        df = Save_Results_to_Drive(results , results_directory)

# -------------------- Plot the error/epoch plot ------------------------

# Define the filename for the Excel file
excel_filename = Plot_directory

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(excel_filename), exist_ok=True)

# Check if the Excel file exists
if os.path.exists(excel_filename):
    # Load existing Excel file
    wb = load_workbook(excel_filename)
    ws = wb.active
else:
    # Create a new Excel workbook
    wb = Workbook()
    ws = wb.active

# Append data to the DataFrame (train_losses, val_losses, train_accuracies, val_accuracies are assumed to be lists)
df['Train Loss'] = train_losses
df['Val Loss'] = val_losses
df['Train Accuracy'] = train_accuracies
df['Val Accuracy'] = val_accuracies
num_epochs = train_losses

# Plotting the error/epoch plot
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Losses ResNet50')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracies ResNet50')
plt.legend()

plt.tight_layout()

# Save the plot as an image (e.g., PNG)
plot_image_path = '/content/plot_ResNet50.png'
plt.savefig(plot_image_path)

# Insert the image into the Excel file
img = xlImage(plot_image_path)
ws.add_image(img, 'A1')

# Save the Excel file
wb.save(excel_filename)
# ----------------- End of Plot the error/epoch plot ---------------------

Training with optimizer: Adam, learning rate: 0.0001, batch size: 32


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 59.0MB/s]


start of epoch 1
Both DS: Fold 2, Epoch 1, F1 Score: 0.5574401868901596, Duration: 992.1621336936951 s
Kaggle: Fold 2, Epoch 1, F1 Score: 0.5478076564862577, Duration: 1051.6769261360168 s
Google: Fold 2, Epoch 1, F1 Score: 0.25190325622981813, Duration: 1056.855675458908 s
start of epoch 2
Both DS: Fold 2, Epoch 2, F1 Score: 0.7589248268328456, Duration: 967.622987985611 s
Kaggle: Fold 2, Epoch 2, F1 Score: 0.7487110481712822, Duration: 1026.7985112667084 s
Google: Fold 2, Epoch 2, F1 Score: 0.4398332060096766, Duration: 1032.02219414711 s
start of epoch 3
Both DS: Fold 2, Epoch 3, F1 Score: 0.7898251184538143, Duration: 970.186544418335 s
Kaggle: Fold 2, Epoch 3, F1 Score: 0.7817118921621962, Duration: 1029.0832941532135 s
Google: Fold 2, Epoch 3, F1 Score: 0.4479795689107486, Duration: 1034.7924993038177 s
start of epoch 4
Both DS: Fold 2, Epoch 4, F1 Score: 0.7953833118084467, Duration: 970.853832244873 s
Kaggle: Fold 2, Epoch 4, F1 Score: 0.7864359997209233, Duration: 1030.1764183

ValueError: Length of values (20) does not match length of index (620)